<a href="https://colab.research.google.com/github/Premerie/OctoMLTrader2/blob/main/MachineLearningAndTrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from textblob import TextBlob
import yfinance as yf

# Fetch historical forex data (GBP/USD)
def get_forex_data(ticker, start_date, end_date):
    forex_data = yf.download(ticker, start=start_date, end=end_date)
    return forex_data

# Fetch financial news data for GBP/USD
def get_news_data(currency_pair, start_date, end_date):
    # Assume you have a CSV file with columns 'Date' and 'News' containing news headlines related to GBP/USD
    # You can replace this with your actual dataset
    news_data = pd.read_csv('gbpusd_news.csv')
    news_data['Date'] = pd.to_datetime(news_data['Date'])
    news_data = news_data[(news_data['Date'] >= start_date) & (news_data['Date'] <= end_date)]
    return news_data

# Feature engineering
def create_features(data):
    data['Date'] = data.index
    data['Date'] = pd.to_datetime(data['Date'])
    data['Day'] = data['Date'].dt.day
    data['Month'] = data['Date'].dt.month
    data['Year'] = data['Date'].dt.year
    return data[['Open', 'High', 'Low', 'Volume', 'Day', 'Month', 'Year']]

# Perform sentiment analysis using TextBlob
def analyze_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

# Merge forex and news data
def merge_data(forex_data, news_data):
    merged_data = pd.merge(forex_data, news_data, how='inner', left_on='Date', right_on='Date')
    return merged_data

# Fetch data
currency_pair = 'GBPUSD=X'  # Yahoo Finance symbol for GBP/USD
start_date = '2022-01-01'
end_date = '2024-01-01'
forex_data = get_forex_data(currency_pair, start_date, end_date)
features = create_features(forex_data)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
target = forex_data['Close']
target

Date
2022-01-03    1.352228
2022-01-04    1.348327
2022-01-05    1.353143
2022-01-06    1.355565
2022-01-07    1.353363
                ...   
2023-12-25    1.267700
2023-12-26    1.270487
2023-12-27    1.272718
2023-12-28    1.280082
2023-12-29    1.273399
Name: Close, Length: 520, dtype: float64

In [ ]:

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [ ]:

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define parameter grid for hyperparameter optimization
param_grid = {'n_neighbors': [3, 5, 7, 9],
              'weights': ['uniform', 'distance'],
              'p': [1, 2]}

# Create KNN regressor
knn_model = KNeighborsRegressor()

# Initialize GridSearchCV
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the model with hyperparameter optimization
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print('Best Parameters:', best_params)

# Use the best model for predictions
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test_scaled)

Best Parameters: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}


In [ ]:
predictions

array([1.23699523, 1.22287607, 1.3592736 , 1.17938931, 1.23344251,
       1.26918766, 1.24252427, 1.18375402, 1.07947774, 1.22849322,
       1.12795413, 1.15996868, 1.19021533, 1.2706307 , 1.36124588,
       1.27842487, 1.2431994 , 1.35391698, 1.2511249 , 1.18749036,
       1.20958823, 1.255089  , 1.21276692, 1.16912057, 1.2334672 ,
       1.24327732, 1.29970305, 1.21575937, 1.25660771, 1.20917057,
       1.21654929, 1.24556944, 1.2463999 , 1.3677895 , 1.23906498,
       1.30885654, 1.27488824, 1.11608596, 1.27172467, 1.20343435,
       1.20135256, 1.25516468, 1.31753764, 1.27097558, 1.27463362,
       1.2138022 , 1.14368661, 1.20559242, 1.21388946, 1.15058686,
       1.13352871, 1.20614456, 1.30540422, 1.23770615, 1.25943054,
       1.21685998, 1.24005178, 1.30523477, 1.35282681, 1.30513201,
       1.30720006, 1.2731711 , 1.24352968, 1.20407084, 1.31158746,
       1.22744493, 1.23855357, 1.30431867, 1.15192033, 1.35884089,
       1.30411774, 1.30586311, 1.34946596, 1.36077376, 1.24216